In [ ]:
import sys; sys.path.append('..')
from bending_validation import *
import elastic_rods, sparse_matrices, pickle, scipy, linkage_vis, numpy as np, time
import MeshFEM, mesh
from tri_mesh_viewer import TriMeshViewer, LineMeshViewer

In [ ]:
cs1 = elastic_rods.CrossSection.fromContour('../../examples/cross_sections/custom_profile.msh', 2000, 0.3, scale=0.0002)
cs2 = elastic_rods.CrossSection.fromContour('../../examples/cross_sections/custom_profile_scaled.msh', 2000, 0.3, scale=0.0002)
v = TriMeshViewer(cs1.interior(0.1), wireframe=True)
v.show()

In [ ]:
# Alternative using ellipse cross-section to validate interpolation against python implementation below
cs1 = elastic_rods.CrossSection.construct('Ellipse', 200, 0.3, [0.005, 0.001])
cs2 = elastic_rods.CrossSection.construct('Ellipse', 200, 0.3, [0.02, 0.005])

In [ ]:
mat1 = elastic_rods.RodMaterial(cs1)
mat2 = elastic_rods.RodMaterial(cs2)

In [ ]:
L, a, n = 1, 0.2, 51
perturb = False
r, fixedVars = bendingTestRod(L, a, n)
pts, thetas = initialConfiguration(L, a, n, perturb)
thetas = np.ones(n - 1) * (np.pi / 2)
r.setDeformedConfiguration(pts, thetas)

view = linkage_vis.LinkageViewer(r, width=800, height=600)
view.show()

In [ ]:
view.averagedMaterialFrames = not view.averagedMaterialFrames
view.averagedCrossSections = not view.averagedCrossSections

In [ ]:
with suppress_stdout(): elastic_rods.compute_equilibrium(r, fixedVars=fixedVars)
view.update()

In [ ]:
r.setLinearlyInterpolatedMaterial(mat1, mat2)
view.update()

In [ ]:
# Change to an inhomogeneous cross-section
def materialAtFrac(alpha):
    mat = elastic_rods.RodMaterial()
    mat.setEllipse(200, 0.3, 0.02 * alpha + 0.005 * (1 - alpha), 0.005 * alpha + 0.001 * (1 - alpha))
    return mat
r.setMaterial([materialAtFrac(alpha) for alpha in np.linspace(0, 1, r.numEdges())])

In [ ]:
with suppress_stdout(): elastic_rods.compute_equilibrium(r, fixedVars=fixedVars)
view.update(True)

### Study the behavior of interpolated cross-sections
Determine what types of interpolation fit their properties well.

In [ ]:
cs_name = 'Freeform'
cs1 = elastic_rods.CrossSection.fromContour('../../examples/cross_sections/custom_profile.msh', 2000, 0.3, scale=0.0002)
cs2 = elastic_rods.CrossSection.fromContour('../../examples/cross_sections/custom_profile_scaled.msh', 2000, 0.3, scale=0.0002)

In [ ]:
cs_name = 'Freeform Warp'
cs1 = elastic_rods.CrossSection.fromContour('../../examples/cross_sections/custom_profile_merged.obj', 2000, 0.3, scale=0.0002)
cs2 = elastic_rods.CrossSection.fromContour('../../examples/cross_sections/custom_profile_merged_warped.obj', 2000, 0.3, scale=0.0002)

In [ ]:
cs_name = 'Ellipse'
cs1 = elastic_rods.CrossSection.construct('Ellipse', 200, 0.3, [0.005, 0.001])
cs2 = elastic_rods.CrossSection.construct('Ellipse', 200, 0.3, [0.02, 0.005])

In [ ]:
cs_name = 'Rectangle'
cs1 = elastic_rods.CrossSection.construct('Rectangle', 200, 0.3, [0.005, 0.001])
cs2 = elastic_rods.CrossSection.construct('Rectangle', 200, 0.3, [0.02, 0.005])

In [ ]:
alphas = np.linspace(0, 1, 1000)
mats = [elastic_rods.RodMaterial(elastic_rods.CrossSection.lerp(cs1, cs2, a)) for a in alphas]
properties = ['B11', 'B22', 'I11', 'I22', 'twistingStiffness', 'torsionStressCoefficient']
test_data = {p: [getattr(m, p) for m in mats] for p in properties}

In [ ]:
plt.figure(figsize=(6, 6))
for i, p in enumerate(properties):
    plt.subplot(3, 2, i + 1)
    coefs = np.polynomial.Polynomial.fit(alphas, test_data[p], 10).coef
    plt.bar(range(len(coefs)), coefs)
    plt.yscale('log')
    plt.title(p)
plt.tight_layout()

In [ ]:
import scipy
import scipy.interpolate

from numpy.linalg import norm as la_norm
from collections import defaultdict

norm = lambda x: la_norm(x, ord=np.inf)

fitAnalysis = defaultdict(list)
nFitSamplesRange = range(6, 50)
for nFitSamples in nFitSamplesRange:
    fit_x = np.linspace(0, 1, nFitSamples)
    fit_mats = [elastic_rods.RodMaterial(elastic_rods.CrossSection.lerp(cs1, cs2, a)) for a in fit_x]

    for p in properties:
        groundTruth = test_data[p]
        gtNorm = norm(groundTruth)
        fit_data = [getattr(m, p) for m in fit_mats]
        
        relErrors = []
        for deg in [4]:
            poly = np.polynomial.Polynomial.fit(fit_x, fit_data, deg)
            relErrors.append(norm(poly(alphas) - groundTruth) / gtNorm)

        for deg in range(1, 6):
            spl  = scipy.interpolate.splrep(fit_x, fit_data, k=deg)
            fspl = lambda x: scipy.interpolate.splev(x, spl)
            relErrors.append(norm(fspl(alphas) - groundTruth) / gtNorm)

        fitAnalysis[p].append(relErrors)

for key, vals in fitAnalysis.items():
    fitAnalysis[key] = np.array(vals)

In [ ]:
plt.figure(figsize=(10, 8))
for i, p in enumerate(properties):
    plt.subplot(3, 2, i + 1)
    plt.plot(nFitSamplesRange, fitAnalysis[p][:, 0], label='quartic')
    plt.plot(nFitSamplesRange, fitAnalysis[p][:, 1], label='piecewise linear')
    plt.plot(nFitSamplesRange, fitAnalysis[p][:, 2], label='quadratic spline')
    plt.plot(nFitSamplesRange, fitAnalysis[p][:, 3], label='cubic spline')
    plt.plot(nFitSamplesRange, fitAnalysis[p][:, 4], label='quartic spline')
    plt.plot(nFitSamplesRange, fitAnalysis[p][:, 5], label='quintic spline')
    plt.yscale('log')
    plt.grid()
    plt.title(f'Relative fitting error for {p}')
    plt.xlabel('fitting samples')
    if i == 0: plt.legend(loc='lower left', bbox_to_anchor=(0, 0.15))
plt.tight_layout()
plt.suptitle(cs_name, fontsize=14)
plt.subplots_adjust(top=0.92)

We find that the quintic spline works best across the various examples.
Note that the scipy documentation warns against using even-degree splines for some reason.

In [ ]:
import cross_section_scaling

import importlib
importlib.reload(cross_section_scaling)

csi = cross_section_scaling.CrossSectionInterpolator(cs1, cs2)

In [ ]:
for p in properties + ['stretchingStiffness', 'area', 'youngModulus', 'shearModulus', 'crossSectionHeight']:
    groundTruth = np.array([getattr(m, p) for m in mats])
    print(p, norm(groundTruth - [getattr(csi(a), p) for a in alphas]) / norm(groundTruth))

In [ ]:
for p in properties + ['stretchingStiffness', 'area', 'youngModulus', 'shearModulus', 'crossSectionHeight']:
    print(p, norm((np.array([getattr(m, p) for m in mats]) - [getattr(csi(a), p) for a in alphas])))

In [ ]:
# Visualize an interpolated cross-section
c = csi(0.5)
pts = np.pad(np.array(c.crossSectionBoundaryPts, dtype=np.float32), [(0, 0), (0, 1)])
lv = LineMeshViewer((pts, np.array(c.crossSectionBoundaryEdges, dtype=np.uint32)))
lv.show()